Instalação e importação de módulos

In [1]:
!pip install -q transformers torch accelerate

In [2]:
import os
import torch
import time
from transformers import AutoTokenizer, AutoModelForCausalLM
from IPython.display import clear_output
from google.colab import drive

Antes de prosseguir, verificar se está com a GPU habilitada. A inferência demora demais se estiver usando apenas CPU.

In [3]:
print("GPU disponível?", torch.cuda.is_available())

if not torch.cuda.is_available():
    print("Ative a GPU em Runtime -> Change runtime type -> T4 GPU.")

GPU disponível? True


Carregando o modelo Qwen2.5-Coder-1.5B-Instruct do HuggingFace

In [4]:
model_id = "Qwen/Qwen2.5-Coder-1.5B-Instruct"

print(f"Carregando {model_id}...")
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    dtype=torch.float16,
    device_map="cuda"
)

Carregando Qwen/Qwen2.5-Coder-1.5B-Instruct...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Definições de funções úteis: **Geração de árvore de diretórios a partir do repositório clonado** e **função de inferência** com prompt.

In [5]:
def gerar_arvore_diretorios(caminho_raiz, max_depth=3, ignorar=['.git', 'node_modules', 'dist', 'build', 'coverage', 'venv', '.github', 'assets']):
    tree_str = ""
    root_level = caminho_raiz.count(os.sep)

    for root, dirs, files in os.walk(caminho_raiz):
        dirs[:] = [d for d in dirs if d not in ignorar]
        level = root.count(os.sep) - root_level
        if level > max_depth:
            continue

        indent = ' ' * 4 * level
        tree_str += f"{indent}{os.path.basename(root)}/\n"

        if level < max_depth:
            subindent = ' ' * 4 * (level + 1)
            for f in files[:10]:
                tree_str += f"{subindent}{f}\n"
            if len(files) > 10:
                tree_str += f"{subindent}... (+{len(files)-10} arquivos)\n"

    return tree_str

def extrair_resumo_readmes(repo_path, max_chars=2000):
    readmes_encontrados = ""

    locais_chave = [
        repo_path,
        os.path.join(repo_path, "server"),
        os.path.join(repo_path, "frontend"),
        os.path.join(repo_path, "collector")
    ]

    for pasta in locais_chave:
        caminho_arquivo = os.path.join(pasta, "README.md")

        if os.path.exists(caminho_arquivo):
            try:
                with open(caminho_arquivo, "r", encoding="utf-8", errors="ignore") as f:
                    conteudo = f.read()

                    conteudo = "\n".join([line for line in conteudo.splitlines() if line.strip()])

                    resumo = conteudo[:max_chars]

                    nome_pasta = os.path.basename(pasta)
                    if nome_pasta == os.path.basename(repo_path):
                        nome_pasta = "RAIZ DO PROJETO"

                    readmes_encontrados += f"\n--- CONTEÚDO DO README ({nome_pasta}) ---\n"
                    readmes_encontrados += resumo
                    readmes_encontrados += "\n... (conteúdo truncado para economizar memória)...\n"
            except Exception as e:
                print(f"Erro ao ler {caminho_arquivo}: {e}")

    return readmes_encontrados

def inferir_arquitetura_pela_tree(tree_text, prompt):
    messages = [
        {"role": "system", "content": "Você é um especialista em análise de código e padrões arquiteturais."},
        {"role": "user", "content": prompt}
    ]

    inputs = tokenizer.apply_chat_template(messages, tokenize=True, return_dict=True, return_tensors="pt", add_generation_prompt=True).to(model.device)

    generated_ids = model.generate(
        **inputs,
        max_new_tokens=1024,
        temperature=0.2
    )

    return tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0].split("assistant")[-1].strip()

Clonando repositório objeto da análise

In [6]:
!git clone https://github.com/Mintplex-Labs/anything-llm.git

fatal: destination path 'anything-llm' already exists and is not an empty directory.


Definição do prompt (en/pt)

In [7]:
prompt = """
Analise os artefatos abaixo de um repositório de software:
1. A estrutura de pastas (Tree).
2. Trechos dos arquivos de documentação (READMEs).

TAREFA:
Classifique a arquitetura deste projeto escolhendo padrões arquiteturais que se encaixem a ele
(ex.: Camadas, Pipe-and-Filters, Client-Server, Peer-to-Peer, Microservices, Blockchain, SOA, Publish-Subscribe, Shared-Data, Blackboard).

IMPORTANTE:
Você DEVE fornecer 5 palpites diferentes, com uma porcentagem de confiança para cada um.

Siga ESTRITAMENTE este formato de resposta:

1. [Nome do Padrão] - [XX]%
   Justificativa: [Explicação baseada em pastas específicas]

2. [Nome do Padrão] - [XX]%
   Justificativa: [...]

(Repita até o 5)

--- INÍCIO DA TREE ---
{}
--- FIM DA TREE ---

--- INÍCIO DOS READMES ---
{}
--- FIM DOS READMES ---
"""

Por fim, executamos o modelo e esperamos pelo resultado da análise (aprox. 40s)

In [8]:
repo_path = "./anything-llm"
tree_visual = gerar_arvore_diretorios(repo_path)
readmes_content = extrair_resumo_readmes(repo_path)

print(f"Estrutura capturada (primeiras 10 linhas):\n{'\n'.join(tree_visual.splitlines()[:10])}...\n")

print("Aguardando análise do modelo...")

start = time.perf_counter()
resultado = inferir_arquitetura_pela_tree(tree_visual, prompt.format(tree_visual, readmes_content))
end = time.perf_counter()

tempo_execucao = end-start

clear_output(wait=True)

print("Análise gerada:\n")
print(f"Tempo de execução = {tempo_execucao}s\n")
print(resultado)

Análise gerada:

Tempo de execução = 24.766483065000102s

1. **Camadas** - O projeto está organizado em camadas, onde as funcionalidades são divididas entre diferentes diretórios. Por exemplo, `server` contém a lógica back-end, enquanto `collector` contém a lógica front-end. Isso facilita a manutenção e escalabilidade do sistema.

2. **Pipe-and-Filters** - O projeto usa pipes e filtros para organizar e manipular dados. Por exemplo, o arquivo `utils/collectorApi/index.js` define um pipe para processar dados da API, enquanto o arquivo `utils/collector/processRawText/index.js` define um filtro para processar texto raw.

3. **Client-Server** - O projeto é cliente-servidor, onde o cliente (front-end) envia solicitações ao servidor (back-end). Isso permite a comunicação entre diferentes partes do sistema e a escalabilidade.

4. **Peer-to-Peer** - O projeto não utiliza peer-to-peer para compartilhar dados ou recursos. No entanto, o uso de serviços como Docker e Kubernetes pode ser considerado

Salvar entradas e saídas do Drive

In [10]:
drive.mount('/content/drive')

input_path = '/content/drive/MyDrive/Qwen-analysis/input-Qwen.txt'
output_path = '/content/drive/MyDrive/Qwen-analysis/output-Qwen.txt'

with open(input_path, 'w') as f:
    f.write(tree_visual)
    f.write(readmes_content)

with open(output_path, 'w') as f:
    f.write(f"TEMPO DE EXECUCAO={tempo_execucao}\n")
    f.write(resultado)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
